In [1]:
from pybacktestchain import data_module
from pybacktestchain import broker
from pybacktestchain.data_module import * 
from datetime import date, timedelta, datetime


from my_package_project.data_treatment import *
from my_package_project.graphs  import *
from my_package_project.operations import *




In [2]:
tickers = ['SPY', 'TLT', 'GLD']  # S&P 500 Index, 20+ Year Treasury Bond, Gold (commodities)
data = get_stocks_data(tickers=tickers, start_date='2000-01-01', end_date='2020-12-31').dropna()
# data['Date'] = data['Date'].astype(str).str[:10] #we had an error with non-UTC dates
# data['Date'] = pd.to_datetime(data['Date'])
data

,Date,Open,High,Low,Close,Adj Close,Volume,ticker
0,2000-01-03 00:00:00-05:00,148.250000,148.250000,143.875000,145.437500,92.692947,8164300,SPY
1,2000-01-04 00:00:00-05:00,143.531250,144.062500,139.640625,139.750000,89.068069,8089800,SPY
2,2000-01-05 00:00:00-05:00,139.937500,141.531250,137.250000,140.000000,89.227409,12177900,SPY
3,2000-01-06 00:00:00-05:00,139.625000,141.500000,137.750000,137.750000,87.793419,6227200,SPY
4,2000-01-07 00:00:00-05:00,140.312500,145.750000,140.062500,145.750000,92.892067,8066500,SPY
...,...,...,...,...,...,...,...,...
4052,2020-12-23 00:00:00-05:00,175.100006,176.210007,175.059998,175.649994,175.649994,6542800,GLD
4053,2020-12-24 00:00:00-05:00,175.550003,176.369995,175.509995,176.350006,176.350006,3695400,GLD
4054,2020-12-28 00:00:00-05:00,177.259995,177.910004,175.630005,175.710007,175.710007,7778700,GLD
4055,2020-12-29 00:00:00-05:00,176.250000,176.970001,175.570007,176.350006,176.350006,5983700,GLD


In [3]:
data_module = DataModule(data=data)
info = Information(s = timedelta(data_module.data.size/len(tickers)), data_module=data_module)
t = datetime(2020,12,29)
moments = FirstTwoMoments.compute_information(info, t=t)


In [4]:
riskparityportfolio =  data_treatment.RiskParity.compute_portfolio_riskparity(info, t=t, information_set=moments)
riskparityportfolio

{'GLD': 0.32828250925007957,
 'SPY': 0.16386582198889166,
 'TLT': 0.5078516687610288}

In [5]:
vol = data_treatment.portfolio_volatility(riskparityportfolio,information_set=moments)
vol

28.86544110045446

In [6]:
data_treatment.RiskParity.compute_portfolio_riskparity_voltarget_leverage(info, t=t, information_set=moments, target_volatility=vol, leverage_factor=1)

INFO:root:Original Volatility: 28.86544110045446


{'GLD': 0.32828250925007957,
 'SPY': 0.16386582198889166,
 'TLT': 0.5078516687610288}

In [7]:
visualizer = PortfolioVisualizer(portfolio=riskparityportfolio, information_set=moments)
visualizer.plot_portfolio_weights()


In [8]:
data_treatment.compute_risk_contributions(riskparityportfolio, moments)

{'GLD': 0.333, 'SPY': 0.333, 'TLT': 0.334}

In [9]:
visualizer.plot_risk_allocation_pie()

In [17]:
test = Backtest_up(
    initial_date = datetime(2017, 1, 1),
    final_date = datetime(2020, 12, 31),
    information_class = FirstTwoMoments,
    risk_model=StopLoss,
)

In [18]:
test.run_backtest()

INFO:root:Running backtest from 2017-01-01 00:00:00 to 2020-12-31 00:00:00.
INFO:root:Retrieving price data for universe
/Users/marc/Desktop/anaconda3/envs/pybacktestchain/lib/python3.11/site-packages/pandas/core/frame.py:11211: RuntimeWarning:

Degrees of freedom <= 0 for slice

/Users/marc/Desktop/anaconda3/envs/pybacktestchain/lib/python3.11/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning:

divide by zero encountered in divide

/Users/marc/Desktop/anaconda3/envs/pybacktestchain/lib/python3.11/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning:

invalid value encountered in multiply

INFO:root:Stop loss triggered for GLD at 2017-01-04 00:00:00. Selling all shares.
INFO:root:Stop loss triggered for SPY at 2017-01-04 00:00:00. Selling all shares.
INFO:root:-----------------------------------
INFO:root:Rebalancing portfolio at 2017-01-31 00:00:00
INFO:root:-----------------------------------
INFO:root:Rebalancing portfolio at 2017-02-28 00:00:00
INFO:root:------

INFO:root:Backtest completed. Final portfolio value: 2018960.195526123
INFO:root:Annualized Return: 10.80%
